In [149]:
import pandas as pd
import numpy as np

In [150]:
df=pd.read_csv("weatherAUS.csv")


In [151]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [152]:
from sklearn.impute import SimpleImputer
imp_mean=SimpleImputer(missing_values=np.nan,strategy='mean')
numeric_columns=list(np.where((x.dtypes==np.int64) | (x.dtypes==np.float64))[0])
imp_mean.fit(x.iloc[:,numeric_columns])
x.iloc[:,numeric_columns]=imp_mean.transform(x.iloc[:,numeric_columns]) 
string_columns=list(np.where((x.dtypes!=np.int64) | (x.dtypes!=np.float64))[0])
imp_mean=SimpleImputer(missing_values=np.nan,strategy='most_frequent')
imp_mean.fit(x.iloc[:,string_columns])
x.iloc[:,string_columns]=imp_mean.transform(x.iloc[:,string_columns])

In [153]:
def OneHotEncoderMethod(indices,data):
    from sklearn.compose import ColumnTransformer
    from sklearn.preprocessing import OneHotEncoder
    columnTransformer = ColumnTransformer([('encoder',OneHotEncoder(),indices)],remainder='passthrough')
    return columnTransformer.fit_transform(data)


In [154]:
def LabelEncoderMethod(series):
    from sklearn import preprocessing
    le=preprocessing.LabelEncoder()
    return le.fit_transform(series)

In [155]:
#encoding selection

In [156]:
string_cols  = list(np.where((x.dtypes == np.object))[0])

C:\Users\KIIT\AppData\Local\Temp\ipykernel_19888\473694297.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  string_cols  = list(np.where((x.dtypes == np.object))[0])


In [157]:
string_cols


[0, 1, 7, 9, 10, 21]

In [158]:
#labelEncoding

In [159]:
y=LabelEncoderMethod(y)

In [160]:
  len(pd.unique(x[x.columns[0]]))

3436

In [161]:
col_0=x.columns[0]

In [162]:
col_0


'Date'

In [163]:
x[col_0]=LabelEncoderMethod(x[col_0])

In [164]:
x.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,396,Albury,13.4,22.9,0.6,5.468232,7.611178,W,44.0,W,...,24.0,71.0,22.0,1007.7,1007.1,8.000000,4.50993,16.9,21.8,No
1,397,Albury,7.4,25.1,0.0,5.468232,7.611178,WNW,44.0,NNW,...,22.0,44.0,25.0,1010.6,1007.8,4.447461,4.50993,17.2,24.3,No
2,398,Albury,12.9,25.7,0.0,5.468232,7.611178,WSW,46.0,W,...,26.0,38.0,30.0,1007.6,1008.7,4.447461,2.00000,21.0,23.2,No
3,399,Albury,9.2,28.0,0.0,5.468232,7.611178,NE,24.0,SE,...,9.0,45.0,16.0,1017.6,1012.8,4.447461,4.50993,18.1,26.5,No
4,400,Albury,17.5,32.3,1.0,5.468232,7.611178,W,41.0,ENE,...,20.0,82.0,33.0,1010.8,1006.0,7.000000,8.00000,17.8,29.7,No


In [165]:
def EncodingSelection(x,threshold=10):
    from sklearn.compose import ColumnTransformer
    from sklearn.preprocessing import OneHotEncoder
    string_cols = list(np.where((x.dtypes == np.object))[0])
    one_hot_encoding_indices = []
    for col in string_cols:
        length = len(pd.unique(x[x.columns[col]]))
        if length == 2 or length > threshold:
            x[x.columns[col]]=LabelEncoderMethod(x[x.columns[col]])
        else:
            one_hot_encoding_indices.append(col)
       
    x=OneHotEncoderMethod(one_hot_encoding_indices,x)
    return x
            
        

In [166]:
 x=EncodingSelection(x,10)
    

C:\Users\KIIT\AppData\Local\Temp\ipykernel_19888\2598887645.py:4: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  string_cols = list(np.where((x.dtypes == np.object))[0])


In [167]:
x.shape

(145460, 22)

In [168]:
#feature selection

In [169]:
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, chi2


In [170]:
kbest = SelectKBest(score_func=chi2,k=10)
mms = preprocessing.MinMaxScaler()

In [171]:
x_temp=mms.fit_transform(x)

In [172]:
x_temp=kbest.fit(x_temp,y)

In [173]:
x_temp.scores_

array([1.50514566e+01, 1.58175252e+01, 4.67783039e+01, 1.22836318e+02,
       6.47157312e+02, 1.44422848e+01, 1.03352623e+03, 1.36011637e+02,
       2.87006856e+02, 3.53677471e+01, 2.91689528e+01, 5.58462514e+01,
       5.26833693e+01, 4.82960487e+02, 2.25219988e+03, 1.57728585e+02,
       1.22708551e+02, 1.19373660e+03, 1.41579509e+03, 3.45236543e+00,
       1.72316599e+02, 1.06248786e+04])

In [174]:
for i in np.sort(x_temp.scores_):
    print('{0:.10f}'.format(i))

3.4523654257
14.4422848064
15.0514565951
15.8175251532
29.1689527570
35.3677471243
46.7783038988
52.6833693451
55.8462514403
122.7085513091
122.8363180739
136.0116369921
157.7285852884
172.3165992609
287.0068557768
482.9604872103
647.1573120166
1033.5262335295
1193.7366038538
1415.7950926455
2252.1998838441
10624.8785997130


In [175]:
best_features=np.argsort(x_temp.scores_)[-13:]

In [176]:
best_features

array([16,  3,  7, 15, 20,  8, 13,  4,  6, 17, 18, 14, 21], dtype=int64)

In [177]:
features_to_delete=np.argsort(x_temp.scores_)[:12]

In [178]:
features_to_delete

array([19,  5,  0,  1, 10,  9,  2, 12, 11, 16,  3,  7], dtype=int64)

In [179]:
x=np.delete(x,features_to_delete,axis=1)

In [180]:
x.shape

(145460, 10)

In [181]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [182]:
sc=preprocessing.StandardScaler(with_mean=False)
sc.fit(x_train)
x_train=sc.transform(x_train)
sc.fit(x_test)
x_test=sc.transform(x_test)

In [183]:
#linear regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
model=LinearRegression()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
r2_score(y_test,y_pred)

0.2214544176700992

In [184]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
model=LinearRegression()
poly=PolynomialFeatures(degree=2)
x_train_poly=poly.fit_transform(x_train)
model.fit(x_train_poly,y_train)
x_test_poly=poly.fit_transform(x_test)
y_pred=model.predict(x_test_poly)
r2_score(y_test,y_pred)

-12315977696953.361

In [185]:
from sklearn.tree import DecisionTreeRegressor
model=DecisionTreeRegressor(max_depth=4)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
r2_score(y_test,y_pred)

0.2992480447992102

In [186]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(n_estimators=25,random_state=10)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
r2_score(y_test,y_pred)

0.2920129898646323

In [187]:
from sklearn.ensemble  import AdaBoostRegressor
model=AdaBoostRegressor(random_state=0,n_estimators=100)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
r2_score(y_test,y_pred)

0.10941208494283117